<a href="https://colab.research.google.com/github/mauricio-ms/motor-imagery-convolutional-recurrent-neural-network/blob/master/notebooks/cnn-1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verify if GPU is enabled

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
  raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


Mount Google Drive directory

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Unzip dataset

In [3]:
!tar -xf "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/preprocessed-csv-files.tar.xz"

Install needed packages

In [4]:
!pip install colorlog

In [5]:
import logging
import colorlog


def get_logger(name="__main__", debug=False):
    log_format = (
        "[%(levelname)s] "
        "%(asctime)s - "
        "%(name)s: "
        "%(funcName)s - "
        "%(message)s"
    )
    bold_seq = "\033[1m"
    colorlog_format = (
        f"{bold_seq} "
        "%(log_color)s "
        f"{log_format}"
    )
    colorlog.basicConfig(format=colorlog_format)

    logger = logging.getLogger(name)
    if debug:
        logger.setLevel(logging.DEBUG)
    else:
        logger.setLevel(logging.INFO)

    # Output full log
    fh = logging.FileHandler("app.log")
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter(log_format)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger

In [6]:
import os
import tensorflow as tf
import numpy as np

LOGGER = get_logger("physionet.py")
N_CHANNELS = 64
PREPROCESSED_CSV_FILES_DIR = "preprocessed-csv-files"

In [7]:
def load_data(train_size=0.75, validation_size=None, n_subjects=None, **kwargs):
    LOGGER.info("Loading Physionet dataset ...")
    subjects = np.array(sorted(os.listdir(PREPROCESSED_CSV_FILES_DIR)))
    if n_subjects is not None:
        subjects = subjects[:n_subjects]
    train_subjects, test_subjects = _train_test_split_subjects(subjects, train_size)
    if validation_size is not None:
        train_subjects, validation_subjects = _train_test_split_subjects(train_subjects, 1-validation_size)
        LOGGER.info(f"(Train, Validation, Test) Subjects = "
                    f"({len(train_subjects)}, {len(validation_subjects)}, {len(test_subjects)})")
        return _load_set(train_subjects, **kwargs), \
            _load_set(validation_subjects, **kwargs), \
            _load_set(test_subjects, **kwargs)

    LOGGER.info(f"(Train, Validation) Subjects = ({len(train_subjects)}, {len(test_subjects)})")
    return _load_set(train_subjects, **kwargs), _load_set(test_subjects, **kwargs)


def _train_test_split_subjects(subjects, train_size):
    train_subjects_mask = np.random.rand(len(subjects)) < train_size
    return subjects[train_subjects_mask], subjects[~train_subjects_mask]


def _load_set(subjects, n_readers=5, n_parse_threads=5, batch_size=100,
              convert_to_2d=False, expand_dim=False):
    path_files = [os.path.join(PREPROCESSED_CSV_FILES_DIR, subject, file_name)
                  for subject in subjects
                  for file_name in sorted(os.listdir(os.path.join(PREPROCESSED_CSV_FILES_DIR, subject)))]
    dataset = tf.data.Dataset.list_files(path_files)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers)
    dataset = dataset.map(lambda r: _preprocess(r, convert_to_2d=convert_to_2d,
                                                expand_dim=expand_dim),
                          num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)


def _preprocess(eeg_record, convert_to_2d=False, expand_dim=False):
    # Create the definitions for the columns (channels + label)
    # The empty array tells TensorFlow to raise exception to missing values
    defs = [tf.constant([], dtype=tf.float32)] * (N_CHANNELS + 1)
    fields = tf.io.decode_csv(eeg_record, record_defaults=defs)
    x = _get_features(fields, convert_to_2d=convert_to_2d)
    if expand_dim:
        x = x[..., np.newaxis]
    y = tf.cast(tf.stack(fields[-1:]), tf.int32)
    return x, y


def _get_features(fields, convert_to_2d=False):
    return tf.stack(fields[:-1])

In [15]:
from tensorflow import keras

with tf.device('/device:GPU:0'):
  train_set, validation_set, test_set = load_data(expand_dim=True, 
                                                  validation_size=0.10,
                                                  n_readers=50,
                                                  n_parse_threads=50)

  model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  optimizer = keras.optimizers.Adam(lr=1e-4)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

  [INFO] 2020-08-14 01:54:40,107 - physionet.py: load_data - Loading Physionet dataset ...
  [INFO] 2020-08-14 01:54:40,113 - physionet.py: load_data - (Train, Validation, Test) Subjects = (70, 11, 27)


In [ ]:
LOGGER.info("Starting training CNN 1D model ...")
model.fit(train_set, epochs=300, validation_data=validation_set, verbose=2)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-14 01:54:45,975 - physionet.py: <module> - Starting training CNN 1D model ...


Epoch 1/300
177689/177689 - 3152s - loss: 0.8574 - accuracy: 0.6581 - val_loss: 1.5192 - val_accuracy: 0.5354
Epoch 2/300
177689/177689 - 3054s - loss: 0.9452 - accuracy: 0.6229 - val_loss: 1.7619 - val_accuracy: 0.5083
Epoch 3/300
177689/177689 - 3049s - loss: 1.0351 - accuracy: 0.5942 - val_loss: 1.5974 - val_accuracy: 0.4995
Epoch 4/300
177689/177689 - 3195s - loss: 1.0572 - accuracy: 0.5847 - val_loss: 1.5349 - val_accuracy: 0.4990
Epoch 5/300
177689/177689 - 3250s - loss: 1.0737 - accuracy: 0.5792 - val_loss: 1.5011 - val_accuracy: 0.5327
Epoch 6/300
177689/177689 - 3347s - loss: 1.0890 - accuracy: 0.5738 - val_loss: 1.4373 - val_accuracy: 0.5389
Epoch 7/300
177689/177689 - 3428s - loss: 1.1007 - accuracy: 0.5711 - val_loss: 1.3945 - val_accuracy: 0.5383
Epoch 8/300
177689/177689 - 3189s - loss: 1.1052 - accuracy: 0.5696 - val_loss: 1.4562 - val_accuracy: 0.5276
Epoch 9/300
177689/177689 - 2994s - loss: 1.1322 - accuracy: 0.5626 - val_loss: 1.4474 - val_accuracy: 0.5359
Epoch 10/3